# <center>Đồ án : Linear Regression</center>

# Import

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
# Import thư viện

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)
train.columns

Index(['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree',
       'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant',
       'Domain', 'ComputerProgramming', 'ElectronicsAndSemicon',
       'ComputerScience', 'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg',
       'CivilEngg', 'conscientiousness', 'agreeableness', 'extraversion',
       'nueroticism', 'openess_to_experience', 'Salary'],
      dtype='object')

# Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` 

In [4]:

features = ['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant', 'Domain']
linear=linear_model.LinearRegression()
linear.fit(train[features],y_train)

w=linear.coef_
b=linear.intercept_
w,b


(array([-23183.32950765,    702.76679172,   1259.0187879 , -99570.60814074,
         18369.9624496 ,   1297.53200035,  -8836.727123  ,    141.75993906,
           145.74234652,    114.64331342,  34955.75040521]),
 49248.08973481221)

In [5]:
# Gọi hàm MAE 
y_pred = linear.predict(test[features])
# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.3f}")

MAE: 105052.530


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\begin{array}{l}
\text{Salary} =49248.090  - 23183.330*Gender + 702.767*10percentage + 1259.019*12percentage - 99570.608*CollegeTier\\
\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\, +  18369.962*Degree + 1297.532*collegeGPA  - 8836.727*CollegeCityTier + 141.760*English\\
\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\, + 145.742*Logical + 114.643*Quant + 34955.750* Domain 
\end{array}$$

# Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất 

Khi sử dụng cross-validation, xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [6]:
personality_features = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']
train_b = pd.read_csv('train.csv')
train_data_shuffled = train_b.sample(frac=1, random_state=42)
mae_scores = []

mae_scores = []

data = []
for i, feature in enumerate(personality_features):
    X = train_data_shuffled[[feature]]
    y = train_data_shuffled['Salary']
    
    model = LinearRegression()
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    mae = -cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error').mean()
    mae_scores.append(mae)
    
    data.append([i+1, feature, mae])

df = pd.DataFrame(data, columns=["STT", "Mô hình với 1 đặc trưng", "MAE"])
styled_df = df.style.hide(axis="index")
display(styled_df)


STT,Mô hình với 1 đặc trưng,MAE
1,conscientiousness,124025.766596
2,agreeableness,123515.372143
3,extraversion,123696.782652
4,nueroticism,123472.421732
5,openess_to_experience,123713.990221


In [7]:
best_feature_index = mae_scores.index(min(mae_scores))
best_feature = personality_features[best_feature_index]
print(f"\nBest Feature: {best_feature}")


Best Feature: nueroticism


In [8]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X_best = train_b[[best_feature]]
y_best = train_b['Salary']

model_best = LinearRegression()
model_best.fit(X_best, y_best)

best_coefficient = model_best.coef_[0]
best_intercept = model_best.intercept_



In [9]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
X_test_best = test[[best_feature]]
y_test_pred = model_best.predict(X_test_best)

mae_test = mean_absolute_error(test['Salary'], y_test_pred)
print(f"MAE on Test Data with Best Personality Feature: {mae_test:.3f}")


MAE on Test Data with Best Personality Feature: 119361.917


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 304647.553 - 16021.494* nueroticism$$

# Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất 

In [10]:
features_skill = ['English', 'Logical', 'Quant']
# Xáo trộn dữ liệu một lần duy nhất
train_data_shuffled = train.sample(frac=1, random_state=42)
# Tìm ra đặc trưng tốt nhất
# Tính toán MAE bằng k-fold Cross Validation cho từng đặc trưng
mae_scores = []


for i, feature in enumerate(features_skill):
    X = train_data_shuffled[[feature]]
    y = train_data_shuffled['Salary']
    
    model = LinearRegression()
    
    # Sử dụng k-fold Cross Validation để tính MAE
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    mae = -cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error').mean()
    mae_scores.append(mae)
    
    
best_skill_feature_index = mae_scores.index(min(mae_scores))
best_skill_feature = features_skill[best_skill_feature_index]
print(f"\nBest Skill Feature: {best_skill_feature}")




Best Skill Feature: Quant


In [11]:
# In ra các kết quả cross-validation 
data = []
for i, feature in enumerate(features_skill):
    data.append([i+1, feature, mae_scores[i]])

df = pd.DataFrame(data, columns=["STT", "Feature", "MAE"])
df

,STT,Feature,MAE
0,1,English,120699.168730
1,2,Logical,120026.708549
2,3,Quant,117343.403934


In [12]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X_best_skill = train_data_shuffled[[best_skill_feature]]
y_best_skill = train_data_shuffled['Salary']

model_best_skill = LinearRegression()
model_best_skill.fit(X_best_skill, y_best_skill)

best_skill_coefficient = model_best_skill.coef_[0]
best_skill_intercept = model_best_skill.intercept_


In [13]:
# Gọi hàm MAE trên tập kiểm tra với mô hình best_skill_feature_model
X_test_best_skill = test[[best_skill_feature]]
y_test_pred_best_skill = model_best_skill.predict(X_test_best_skill)

mae_test_best_skill = mean_absolute_error(test['Salary'], y_test_pred_best_skill)
print(f"\nMAE on Test Data with Best Skill Feature: {mae_test_best_skill:.3f}")


MAE on Test Data with Best Skill Feature: 108814.060


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 117759.729 + 368.852*Quant$$

# Tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất

## Tìm mô hình

In [35]:
# Trình bày các phần tìm ra mô hình
def best_multifeature(train, k_fold=5):
    best_mae = float('inf')
    best_features = None
    mae_arr = []

    # Định nghĩa feature_sets
    feature_sets = [
        ["Quant",  "10percentage", "12percentage", "English", "ComputerProgramming", "ComputerScience"],
        ["Quant",  "12percentage", "English","ComputerProgramming", "ComputerScience" ],
        ["Quant",  "12percentage","ComputerProgramming", "ComputerScience" ]
    ]

    for features in feature_sets:
        X = train[features]
        y = train['Salary']

        mae_scores = []

        kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)
            mae_scores.append(mae)

        avg_mae = sum(mae_scores) / k_fold
        mae_arr.append(avg_mae)

        if avg_mae < best_mae:
            best_mae = avg_mae
            best_features = features

    return best_mae, best_features, mae_arr
   

## Thử nghiệm, so sánh các mô hình

In [36]:

best_mae, best_features, mae_arr = best_multifeature(train, k_fold=5)
feature_sets = [
    ["Quant",  "10percentage", "12percentage", "English", "ComputerProgramming", "ComputerScience"],
    ["Quant",  "12percentage", "English","ComputerProgramming", "ComputerScience" ],
    ["Quant",  "12percentage","ComputerProgramming", "ComputerScience" ]
]

# In ra các kết quả cross-validation
print("STT\tMô hình\t\t\tMAE")
for idx, mae in enumerate(mae_arr):
    features_used = feature_sets[idx]
    print(f"{idx+1}\tSử dụng {len(features_used)} đặc trưng {features_used}\t{mae:.3f}")
print("Best Features:", best_features)
print("Best MAE:", best_mae)

# In ra thông tin mô hình tốt nhất
best_features_names = best_features
print("Đặc trưng được chọn:", best_features_names)



STT	Mô hình			MAE
1	Sử dụng 6 đặc trưng ['Quant', '10percentage', '12percentage', 'English', 'ComputerProgramming', 'ComputerScience']	111474.994
2	Sử dụng 5 đặc trưng ['Quant', '12percentage', 'English', 'ComputerProgramming', 'ComputerScience']	111879.040
3	Sử dụng 4 đặc trưng ['Quant', '12percentage', 'ComputerProgramming', 'ComputerScience']	113072.880
Best Features: ['Quant', '10percentage', '12percentage', 'English', 'ComputerProgramming', 'ComputerScience']
Best MAE: 111474.99424251467
Đặc trưng được chọn: ['Quant', '10percentage', '12percentage', 'English', 'ComputerProgramming', 'ComputerScience']


In [37]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

X_train_best = train[best_features]
y_train_best = train['Salary']
model = LinearRegression()
model.fit(X_train_best, y_train_best)
X_test_best = test[best_features]
y_test_best = test['Salary']

In [38]:
# Gọi hàm MAE trên tập kiểm tra với mô hình my_best_model

y_pred_best = model.predict(X_test_best)
mae_best = mean_absolute_error(y_test_best, y_pred_best)
print(f"MAE : {mae_best:.3f}")


MAE : 103143.290


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -86901.354 + 213.672 * Quant + 965.629 * 10percentage + 974.106 * 12percentage + 207.983 * English \\
+ 136.923 * ComputerProgramming - 164.443 * ComputerScience   $$